# Add Folders to Path

In [1]:
%%time
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add integration folder to current working path
sys.path.append(parent_directory+"/Src/integration")

# add utils folder to current working path
sys.path.append(parent_directory+"/Src/utils")

# add Algorithm folder to current working path in order to access the functions
sys.path.append(parent_directory+"/Demos/HyperbolicOECS")

CPU times: user 151 µs, sys: 137 µs, total: 288 µs
Wall time: 194 µs


In [2]:
# Suppress numba warnings arising from computation falling back into object mode.
from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)

# Overview

In the following notebok we extract elliptic OECS in the Agulhas region from the AVISO dataset. The notebook is structured as follows:

* Import data from the file 'Agulhas_AVISO.mat' stored in the folder 'Data'
<br />
* Define computational parameters (such as the number of cores) and variables
 <br />
* Define spatio-temporal domain over which to compute the elliptic OECS.
 <br />
* Interpolate velocity from the (discrete) gridded data using a cubic spline interpolation <br />
* Compute rate of strain tensor $ \mathbf{S}(\mathbf{x}, t) $ over meshgrid  <br />
* Find objective saddle points
* Compute hyperbolic OECS as tensorlines tangent to the eigenvectors $ \mathbf{e_1}(\mathbf{x}), \mathbf{e_2}(\mathbf{x}) $ of the rate of strain tensor.

# Import Agulhas data

In [3]:
%%time
import scipy.io as sio

#Import velocity data from file in data-folder
mat_file = sio.loadmat('../../../Data/AVISO/Agulhas_AVISO.mat')

#U = mat_file['u']
#V = mat_file['v']
#x = mat_file['x']
#y = mat_file['y']
#time = mat_file['t']

mat_file = sio.loadmat('../../../Data/AVISO/ocean_velocity_example_diffusive.mat')

U = mat_file['vx'].transpose()
V = mat_file['vy'].transpose()
x = mat_file['xc'].transpose()
y = mat_file['yc'].transpose()
time_data = mat_file['time'].transpose()

CPU times: user 35.1 ms, sys: 15.2 ms, total: 50.3 ms
Wall time: 49.1 ms


# Data/Parameters for Dynamical System

In [4]:
import numpy as np

# number of cores to be used for parallel computing
Ncores = 8

# time resolution of data
dt_data = time_data[0, 1]-time_data[0,0]

# periodic boundary conditions
periodic_x = False
periodic_y = False
periodic = [periodic_x, periodic_y]

# unsteady velocity field
bool_unsteady = True

# defined domain
defined_domain = np.isfinite(U[:,:,0]).astype(int)

## compute meshgrid of dataset
X, Y = np.meshgrid(x, y)

## resolution of meshgrid
dx_data = X[0,1]-X[0,0]
dy_data = Y[1,0]-Y[0,0]

delta = [dx_data, dy_data]

<a id='Spatio-Temporal-Domain-of-Dynamical-System'></a>
# Spatio-temporal domain of dynamical system

In [6]:
%%time
# Time
t_OECS = 0 

# store time in array
time = np.array([t_OECS])

# domain boundary (in degrees)
xmin = -3
xmax = 7
ymin = -38
ymax = -28

# spacing of meshgrid (in degrees)
dx = 0.01
dy = 0.01

x_domain = np.arange(xmin, xmax + dx, dx)
y_domain = np.arange(ymin, ymax + dy, dy)

X_domain, Y_domain = np.meshgrid(x_domain, y_domain)

CPU times: user 3.23 ms, sys: 4.45 ms, total: 7.68 ms
Wall time: 6.22 ms


# Interpolate Velocity

In order to evaluate the velocity field at arbitrary locations and times, we must interpolate the discrete velocity data. The interpolation with respect to time is always linear. The interpolation with respect to space can be chosen to be "cubic" or "linear". In order to favour a smooth velocity field, we interpolate the velocity field in space using a cubic interpolant. 

In [ ]:
%%time
# Import interpolation function for unsteady flow field
from ipynb.fs.defs.Interpolant import interpolant_unsteady

# Interpolate velocity data using cubic spatial interpolation
Interpolant = interpolant_unsteady(X, Y, U, V, time_data, method = "cubic")

# Rate of strain tensor over meshgrid of initial conditions

The rate of strain tensor $ S(\mathbf{x}, t) $ at time $ t $ is computed by iterating over meshgrid. The method *DS._spin_tensor(x, t)* computes the rate of strain tensor at point $ \mathbf{x} $  at time $ t $ by using an auxiliary meshgrid. 'aux_grid' specifies the ratio between the auxiliary grid and the original meshgrid. This parameter is generally chosen to be between $ [\dfrac{1}{10}, \dfrac{1}{100}] $. The computations are parallelized.

In [ ]:
%%time
# Import package for progress bar
from tqdm.notebook import tqdm

# Import package for parallel computing
from joblib import Parallel, delayed

# Import gradient of velocity function
from ipynb.fs.defs.gradient_velocity import gradient_velocity

# Import Rate of Strain function
from ipynb.fs.defs.CauchyGreen import CauchyGreen

# Define ratio of auxiliary grid spacing vs original grid_spacing
aux_grid_ratio = .1 # [1/10, 1/100]
aux_grid = [aux_grid_ratio*(X_domain[0, 1]-X_domain[0, 0]), aux_grid_ratio*(Y_domain[1, 0]-Y_domain[0, 0])]

def parallel_S(i):
    
    S_parallel = np.zeros((X_domain.shape[1], 2, 2))
    
    for j in range(S_parallel.shape[0]):
        
        x = np.array([X_domain[i,j], Y_domain[i,j]])
        
        # Compute gradient of velocity
        grad_vel = gradient_flowmap(t_OECS, x, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta, aux_grid)
        
        # Compute rate of strain at 'x'
        S_parallel[j, :, :] = RateStrain(grad_vel)
    
    return S_parallel

S = np.array(Parallel(n_jobs=Ncores, verbose = 0)(delayed(parallel_S)(i) for i in tqdm(range(X_domain.shape[0]))))

# Eigenvalues/Eigenvectors of rate of strain tensor

We now compute the properties of the rate of strain tensor 'S' such as the eigenvalues 's1', 's2' and eigenvectors 'eigenv1', 'eigenv2'. Furthermore, we also need the spatial derivatives of the elements of 'S'.

In [ ]:
# Import eigenvalues/eigenvectors calculator
from ipynb.fs.defs.eigen import eigen

eig1 = S[:,:,0,0].copy()*np.nan
eig2 = S[:,:,0,0].copy()*np.nan
e1 = np.zeros((S.shape[0], S.shape[1], 2))
e2 = np.zeros((S.shape[0], S.shape[1], 2))

#iterate over meshgrid
for i in range(X_domain.shape[0]):
    
    for j in range(Y_domain.shape[1]):
            
        x = [X_domain[i,j], Y_domain[i, j]]
        
        # compute eigenvalues of rate of strain
        eig1[i,j], eig2[i,j], e1[i,j,:], e2[i,j,:] = eigen(S[i,j,:,:])

# Find objective Saddle-points

In [ ]:
# import function to compute local maxima
from ipynb.fs.defs.loc_max import _loc_max

# minimum distance between objective saddles (=local maxima in the s2-field)
min_distance = .1

# indices, positions and value (of s2-field) of the objective saddles
loc_idx_x, loc_idx_y, loc_max_x, loc_max_y, loc_max_field = _loc_max(min_distance, X_domain, Y_domain, eig2)

In [ ]:
###################################### PLOT OBJECTIVE SADDLES ######################################
import matplotlib.pyplot as plt

fig = plt.figure(figsize = (16, 10))
ax = plt.axes()

cax = ax.contourf(X_domain, Y_domain, eig2, levels = 400, cmap = "Blues")

ax.scatter(loc_max_x, loc_max_y, c = 'y', marker = "X", s = 100)
    
ax.set_xlabel('long', fontsize = 16)
ax.set_ylabel('lat', fontsize = 16)

ax.set_xticks(np.arange(np.min(X_domain), np.max(X_domain), 1))
ax.set_yticks(np.arange(np.min(Y_domain), np.max(Y_domain), 2))

ax.set_title('Objective Saddles', fontsize = 20)

cbar = plt.colorbar(cax)
cbar.set_ticks(np.arange(0, 1.5, 0.1))
cbar.set_label(r'$ |s| $', rotation = 0, labelpad = 20, fontsize = 16)

plt.show()

## Shrinklines (Repelling OECS)

Repelling LCS can be sought among trajectories of the differential equation:

\begin{equation}
\mathbf{x}'_0(s) = \xi_1(\mathbf{x}_0;t),
\label{eq: shrinklines}
\end{equation}

with $ \xi_1 $ denoting the eigenvector associated to the weakest eigenvalue $ s_1 $ of $ S(\mathbf{x},t) $. The non-orientable vector field is well defiend away from tensorline singularites (points where $ s_1 = s_2 $).
The most repelling shrinklines mark initial positions of repelling LCSs. Repelling LCSs can therefore be located as trajectories of eq. \ref{eq: shrinklines} that have locally the largest averaged $ s_2(\mathbf{x},t) $ among all neighbouring shrinklines.

In [ ]:
from ipynb.fs.defs.tensorlines import _tensorlines

# step-size used for integration with respect to parameter 's'
step_size = 0.01

# threshold distance to locate local maxima in the 'eig2'-field
max_distance = 0.5

# maximum length of shrinkline
max_length = 3

# Minimum threshold on rate of repulsion of shrinkline 
hyperbolicity = .2

shrinklines = _tensorlines(X_domain, Y_domain, eig2, e1, max_distance, max_length, step_size, hyperbolicity)

## Stretchlines (Attracting OECS)

Attracting LCS can be sought among trajectories of the differential equation:

\begin{equation}
\mathbf{x}'_0 = \xi_2(\mathbf{x}_0;t),
\label{eq: stretchlines}
\end{equation}

with $ \xi_2 $ denoting the eigenvector associated to the strongest eigenvalue $ s_2 $ of $ S(\mathbf{x},t) $. The non-orientable vector field is well defiend away from tensorline singularites (points where $ s_1 = s_2 $).
The most attracting stretchlines mark initial positions of attracting LCSs. Attracting OECSs can therefore be located as trajectories of eq. \ref{eq: stretchlines} that have locally the smallest averaged $ s_1(\mathbf{x},t) $ among all neighbouring stretchlines.

In [ ]:
from ipynb.fs.defs.tensorlines import _tensorlines

# step-size used for integration with respect to parameter 's'
step_size = 0.01

# threshold distance to locate local minima in the 's_1' field of 'S'
max_distance = 0.5

# maximum length of stretchline
max_length = 3

# Minimum threshold on rate of attraction of stretchline 
hyperbolicity = 0.2

stretchlines = _tensorlines(X_domain, Y_domain, eig2, e2, max_distance, max_length, step_size, hyperbolicity)

In [ ]:
################################ PLOT HYPERBOLIC OECS ################################

import matplotlib.pyplot as plt

# generate figure and axis object
fig = plt.figure(figsize = (16, 12))
ax = plt.axes()

# Plot eig2-field
cax = ax.contourf(X_domain, Y_domain, eig2, levels = 600, cmap = "gist_rainbow_r")

cbar = plt.colorbar(cax)
cbar.set_ticks(np.arange(0, 1.5, 0.1))
cbar.set_label(r'$ |s_1| $', rotation = 0, labelpad = 20, fontsize = 16)

# Plot attracting OECS  
for i in range(len(stretchlines[0])):
    ax_attracting = ax.plot(stretchlines[0][i], stretchlines[1][i], c = 'k', linewidth = 5)

# Plot repelling OECS
for j in range(len(shrinklines[0])):
    ax_repelling = ax.plot(shrinklines[0][j], shrinklines[1][j], c = "w", linewidth = 5)

# set limits
ax.set_xlim(np.min(X_domain), np.max(X_domain))
ax.set_ylim(np.min(Y_domain), np.max(Y_domain))

# Set title
ax.set_title('Repelling/Attracting OECSs following Serra', fontsize = 20)

# Plot legend
import matplotlib.patches as mpatches
attracting = mpatches.Patch(color='k', label='attracting OECS')
repelling = mpatches.Patch(color='w', label='repelling OECS')
plt.legend(handles=[attracting, repelling], fontsize = 16,  loc = "lower left")
 
# Show plot
plt.show()